# Olivetti con clasificadores discriminativos

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from sklearn.datasets import fetch_olivetti_faces
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import make_pipeline

In [2]:
orl = fetch_olivetti_faces()
X = orl.data
y = orl.target
X_train, X_test, y_train, y_test = train_test_split(
    orl.data, orl.target, test_size=0.3, shuffle=True, random_state=23)

Perceptrón con penalización L2 en función de alpha:

In [3]:
for a in np.logspace(-8,-3,6):
    clf = Perceptron(penalty='l2', alpha=a, random_state=0).fit(X_train, y_train)
    acc = accuracy_score(y_test, clf.predict(X_test))
    print('La precisión de {0!s} es {1:.1%}'.format(clf, acc))

La precisión de Perceptron(alpha=1e-08, penalty='l2') es 89.2%
La precisión de Perceptron(alpha=1e-07, penalty='l2') es 89.2%
La precisión de Perceptron(alpha=1e-06, penalty='l2') es 88.3%
La precisión de Perceptron(alpha=1e-05, penalty='l2') es 85.8%
La precisión de Perceptron(penalty='l2') es 87.5%
La precisión de Perceptron(alpha=0.001, penalty='l2') es 57.5%


Regresión logística con varios solvers (y max_iter=10000).

In [4]:
for solver in ['lbfgs', 'liblinear', 'newton-cg', 'sag', 'saga']:
    clf = LogisticRegression(solver=solver, max_iter=10000).fit(X_train, y_train)
    acc = accuracy_score(y_test, clf.predict(X_test))
    print('La precisión de {0!s} es {1:.1%}'.format(clf, acc))

La precisión de LogisticRegression(max_iter=10000) es 98.3%
La precisión de LogisticRegression(max_iter=10000, solver='liblinear') es 97.5%
La precisión de LogisticRegression(max_iter=10000, solver='newton-cg') es 98.3%
La precisión de LogisticRegression(max_iter=10000, solver='sag') es 98.3%
La precisión de LogisticRegression(max_iter=10000, solver='saga') es 98.3%


Dado que tenemos $4097$ características originales (en notación homogénea), podríamos añadir $4096\cdot 4097/2=8390656$ características cuadráticas. Claramente, regresión logística con características cuadráticas no es factible por su elevado coste computacional. Así pues, trataremos de mejorar la precisión de regresión logística mediante ingeniería de características basada en estandarización (StandardScaler) y reducción de la dimensión (PCA).

In [5]:
def lr_exp(standardize=True, degree=1, n_components=0):
    clf = make_pipeline(StandardScaler() if standardize else None, 
    PCA(n_components=n_components) if n_components > 0 else None,
    LogisticRegression(max_iter=10000, solver='saga', n_jobs=8)).fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return accuracy_score(y_test, y_pred)

In [6]:
for n_components in [8, 16, 32, 64, 128, 0]:
    for standardize in [False, True]:
        acc = lr_exp(standardize=standardize, degree=2, n_components=n_components)
        print('standardize {0:} components {1:}: {2:.1%} acc'.format(standardize, n_components, acc))

standardize False components 8: 77.5% acc
standardize True components 8: 77.5% acc
standardize False components 16: 92.5% acc
standardize True components 16: 93.3% acc
standardize False components 32: 96.7% acc
standardize True components 32: 96.7% acc
standardize False components 64: 97.5% acc
standardize True components 64: 99.2% acc
standardize False components 128: 98.3% acc
standardize True components 128: 98.3% acc
standardize False components 0: 98.3% acc
standardize True components 0: 99.2% acc
